In [2]:
# Basic ReadMe
# for Computer Vision final project!
#
# For that project, several pictures being converted and rotated,
# All steps include morpological operations from cv2,
# such as delation, erosion, edges and others.
#
# Numpy was using for kernels,
# to improve there work
#
# After you run program,
# a picture of hand should appear.
# It's important to chose sector !!!
# WITH 4 FINGERS WE NEED
# Put a square with mouse,
# Press "Enter"
#
# Then Several Outputs should be presented!
#
# During the lines of code,
# More explanation,
# Will be present!
#
# DataSpell Python 3.10
# After Program present
# All output
# Press Enter and it finished!
#
# !@1 Project by Artur Bruev
#
# (basic review was sending a team,
# to improve there work and understand better a question)
#
#
# settings for 001_norm.jpg
# th1 = 100, th2 = 180
#
# settings for 001_norm_rotated.jpg
# th1 = 100, th2 = 180
#
# settings for 002_norm.jpg
# th1 = 50, th2 = 180
#
# settings for 002_norm_rotated.jpg
# th1 = 50, th2 = 180
#
#
#
# Here starts realization!

In [3]:
import cv2
import numpy as np

In [4]:
path = "002_norm_rotated.jpg"

In [5]:
# Settings of gradient intensivity to find hand lines
# different pictures should consider different settings
# it's much better to use correct 'rotate' pictures

th1 = 50
th2 = 180

# We can raise th2,
# until all unimportant white objects,
# will disappear.

In [6]:
# read image and finding future shape for drawing

colored = cv2.imread(path)
shape_init = colored.shape

In [7]:
# resize, hand should be on same position always
# using same size pictures to better identification

colored = cv2.resize(colored, (1000, 1000))

In [8]:
# identify region

roi = cv2.selectROI("Select reg", colored)

# read same image in GrayScale, for future operations

initial_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
initial_image = cv2.resize(initial_image, (1000, 1000))

In [9]:
# kernel for operation

kernel1 = np.array([[1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, -50, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1]])

# kernel for dilation

kernel3 = np.array([[0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0]], np.uint8)

# kernel for erosion
kernel4 = np.array([[0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 1, 1, 1, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0]], np.uint8)

# kernel for morphological operations, in elipse
kernel5 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (17, 17))

In [10]:
# realise operations and finding edges

identity1 = cv2.filter2D(initial_image, ddepth=-1, kernel=kernel1)
edges = cv2.Canny(image=initial_image, threshold1=th1, threshold2=th2)
erosion = cv2.erode(edges, kernel4, iterations=1)

dilation1 = cv2.dilate(erosion, kernel3, iterations=2)
dilation2 = cv2.dilate(dilation1, kernel5, iterations=2)

In [11]:
# finding contours

contours, hierarchy = cv2.findContours(dilation2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [12]:
# create empty mask, to save sort shapes

mask = np.ones(edges.shape[:2], dtype="uint8") * 0

In [13]:
for c in contours:                          # for every corner
    area = cv2.contourArea(c)               # if S of object is in zone
    if 1000 < area < 10000:                 # supposed on change of this parameter,
        hull = cv2.convexHull(c)            # output will be more or less usefull for operation
    rect = cv2.minAreaRect(hull)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    M = cv2.moments(box)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    if int(roi[0]) < cx < int(roi[0] + roi[2]) and int(roi[1]) < cy < int(roi[1] + roi[3]):        # if center of box in region
        cv2.drawContours(mask, [hull], -1, 255, -1)                                                # put box on mask
        cv2.drawContours(colored, [box], -1, (0, 0, 255), 1)                                       # on color picture draw box around white zones
        colored = cv2.circle(colored, (cx, cy), 5, (0, 0, 255), -1)                                # finding center of box and centralise it

colored = cv2.resize(colored, (shape_init[1], shape_init[0]))                                      # change size of picture to back

In [14]:
# Present all Outputs

cv2.imshow('resultant mask', mask)
cv2.imshow('edges', edges)
cv2.imshow('dilate', dilation2)
cv2.imshow('total', colored)

# cv2.imshow('erosion', erosion) # in case

cv2.imwrite('res.jpg', colored)

True

In [15]:
cv2.waitKey(0)

13

In [16]:
# Enter to quit // Output presentation finished
quit(0)